#### 요일변수 생성, 컬럼명 단순화, 불필요 컬럼 제거 함수, 유성구 대청동 데이터 -> 진잠동으로 처리

In [ ]:
import pandas as pd
import locale

df = pd.read_csv("D:/치안빅데이터/데이터/총인구수추가 완.csv", encoding='euc-kr')

def model(df):
    df['접수완료날짜'] = pd.to_datetime(df['접수완료날짜'])
    df['요일'] = df['접수완료날짜'].dt.day_name()
    
    locale.setlocale(locale.LC_ALL, 'ko_KR.UTF-8')
    df['요일'] = df['접수완료날짜'].dt.strftime('%A')
    
    col = ['사건종별','발생지점','발생좌표x','발생좌표y','접수완료날짜','접수완료시간대','접수연도','접수월','접수일자',
       '접수완료시간', '시군구', '행정동', '면적(km^2)', '기온(°C)', '강수량(mm)', '적설(cm)', '총인구수', '요일']

    df = df[col]

    df.rename(columns={'면적(km^2)':'면적',
                           '기온(°C)':'기온',
                           '강수량(mm)':'강수량',
                           '적설(cm)':'적설'},inplace=True)
    
    df.loc[(df['시군구'] == '유성구') & (df['행정동'] == '대청동'),'행정동'] = '진잠동' #진잠동 처리

    return(df)

df = model(df)

### 가설: 사건종별로 발생횟수에는 차이가 있다 
#### 위 데이터프레임을 분산분석해서 사건종별에 따라 발생횟수에 차이가 있는지 없는지 검정해보자

In [ ]:
df_new = df.groupby(['사건종별','시군구','행정동']).size().reset_index(name='발생횟수').sort_values(by='발생횟수',ascending=False)
a = df_new['사건종별'] != 404 # 소수 데이터
b = df_new['사건종별'] != 405 # 소수 데이터

df_new[a&b]

In [ ]:
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

df_401 = df_new[df_new['사건종별'] == 401]
df_402 = df_new[df_new['사건종별'] == 402]
df_403 = df_new[df_new['사건종별'] == 403]
df_406 = df_new[df_new['사건종별'] == 406]

stat, p = stats.normaltest(df_401['발생횟수'])
print("401: Statistics=%.3f, p=%.4f" % (stat, p))
stat, p = stats.normaltest(df_402['발생횟수'])
print("402: Statistics=%.3f, p=%.4f" % (stat, p))
stat, p = stats.normaltest(df_403['발생횟수'])
print("403: Statistics=%.3f, p=%.4f" % (stat, p))
stat, p = stats.normaltest(df_406['발생횟수'])
print("406: Statistics=%.3f, p=%.4f" % (stat, p))

#### 네 그룹 모두 정규성 가정 불만족 -> 분산분석 대신 kruskal 순위합 검정 사용

In [ ]:
from scipy.stats import kruskal

df_new_401 = df_new[df_new['사건종별'] == 401]['발생횟수']
df_new_402 = df_new[df_new['사건종별'] == 402]['발생횟수']
df_new_403 = df_new[df_new['사건종별'] == 403]['발생횟수']
df_new_406 = df_new[df_new['사건종별'] == 406]['발생횟수']

stat, p = kruskal(df_new_401, df_new_402, df_new_403, df_new_406)

print('검정통계량:',stat)
print('p-value:',p)
print('-'*50)
if p < 0.05:
    print("H0 reject")
else:
    print("Fail to reject H0")

#### 사건종별에 따른 발생횟수 평균에는 차이가 있다. 즉, 사건종류에 따라서 발생횟수의 평균은 다르다

### 1. 사건종별 발생횟수 예측 
#### 아래 데이터프레임 형태를 각자 뜯어서

#### 사건종별로 발생횟수를 예측할건데, 각 행정동별로, (요일, 강수량, 적설, 시간 등) 여러 변수의 조건 하에서 얼만큼의 사건이 발생할 것이다

#### 따라서 유사한 그룹을 묶어서 이런 지역엔 가령 402번이 자주 일어나기 때문에 이런이런 조치가 필요할 것이다라는 형식으로 마무리

In [ ]:
df4 = df.copy()
df4['강수량'] = df4['강수량'].apply(lambda x: 0 if pd.isnull(x) else 1)
df4['적설'] = df4['적설'].apply(lambda x: 0 if pd.isnull(x) else 1)

In [ ]:
df4 = df4.groupby(['사건종별','접수완료시간','요일','시군구','행정동','총인구수','면적','강수량','적설']).size().reset_index(name='발생횟수')
df4.sort_values(by='발생횟수',ascending=False)

### 2. 발생횟수 다수지점 인사이트 찾기 및 예측
#### 발생횟수 기준, 3사분위수 이상 해당하는 지점들만 데이터프레임으로 뽑아서 가져옴 

In [ ]:
do = pd.DataFrame(df['발생지점'].value_counts())
do.describe()

#### 3사분위수 이상에 해당하는 놈들만 가져오려 했는데, 3사분위수가 2이다...
#### 그래도 일단 가져와보자 

In [ ]:
do = pd.DataFrame(df['발생지점'].value_counts())
do[do['발생지점'] > 2]

In [ ]:
counts = df_col['발생지점'].value_counts()
mask = df_col['발생지점'].isin(counts[counts >= 50].index)
df_filtered = df_col[mask]
df_filtered

#### 해당 데이터프레임을 그룹바이하여 원하는 형태로 만들어서 예측해볼 것